# Practical works 4: Voting rules

Let us consider an election with n voters (0 ≤ n ≤ 100) and m candidates (0 ≤ m ≤ 6). We assume that :

• The preferences of each voter are given as a linear order (total order) on the set of candidates

• All the preferences (of the n voters) are contained in an Excel file or a csv file.

This work aims at computing in python language the voting rules introduced in Chapter 2. You can use the examples of
this chapter to test your functionalities, especially the following example where we have m = 4 candidates {a, b, c, d} and
n = 27 voters :


5 voters : a > b > c > d

4 voters : a > c > b > d

2 voters : d > b > a > c

6 voters : d > b > c > a

8 voters : c > b > a > d

2 voters : d > c > b > a

In [1]:
import csv
import numpy as np
import random
import collections

voters_1 = []

#import data from csv file
with open("data.csv", newline='') as File:
    data = csv.reader(File, delimiter=',')
    for row in data:
        row[4] = int(row[4])
        voters_1.append(row)
        
voters_1

[['a', 'b', 'c', 'd', 5],
 ['a', 'c', 'b', 'd', 4],
 ['d', 'b', 'a', 'c', 2],
 ['d', 'b', 'c', 'a', 6],
 ['c', 'b', 'a', 'd', 8],
 ['d', 'c', 'b', 'a', 2]]

1. Compute a function MajorityRule returning the result of a simple majority rule voting, between two candidates.

In [2]:
def MajorityRule(candidate_1, candidate_2, voters):
    count_1 = 0
    count_2 = 0
    
    #compare amount of votes for given candidates
    for voter in voters:
        for i in voter:
            if i == candidate_1:
                count_1 += voter[len(voter)-1]
                break
            elif i == candidate_2:
                count_2 += voter[len(voter)-1]
                break

                
    if count_1 > count_2:
        return candidate_1
    else:
        return candidate_2

print("Beetween a and b according to Majority rule, candidate -", MajorityRule('a', 'b', voters_1), "is elected")
print("Beetween a and c according to Majority rule, candidate -", MajorityRule('a', 'c', voters_1), "is elected")
print("Beetween a and d according to Majority rule, candidate -", MajorityRule('a', 'd', voters_1), "is elected")
print("Beetween b and c according to Majority rule, candidate -", MajorityRule('b', 'c', voters_1), "is elected")
print("Beetween b and d according to Majority rule, candidate -", MajorityRule('b', 'd', voters_1), "is elected")
print("Beetween c and d according to Majority rule, candidate -", MajorityRule('c', 'd', voters_1), "is elected")

Beetween a and b according to Majority rule, candidate - b is elected
Beetween a and c according to Majority rule, candidate - c is elected
Beetween a and d according to Majority rule, candidate - a is elected
Beetween b and c according to Majority rule, candidate - c is elected
Beetween b and d according to Majority rule, candidate - b is elected
Beetween c and d according to Majority rule, candidate - c is elected


2. Compute a function Plurality returning the result of a plurality voting.

In [3]:
def Plurality(arr_candidates, voters):
    
    dictionary = {}
    count = [0] * len(arr_candidates)
    b = False
    
    for voter in voters:
        for i in voter:
            for candidate in arr_candidates:
                #sum all votes for one candidate
                if i == candidate:
                    count[arr_candidates.index(candidate)] += voter[len(voter)-1]
                    b = True
                    
            if b == True:
                break
    #create dictionary with names of the candidates and sum of their votes        
    for candidate in arr_candidates:
        dictionary[count[arr_candidates.index(candidate)]] = candidate
    
    #return candidate name with maximum value of votes
    winner = dictionary.get(max(count))
    return winner

print("According to Plurality rule, candidate -", Plurality(['a', 'b', 'c', 'd'],voters_1), "is elected")

According to Plurality rule, candidate - d is elected


3. Compute a function PluralityRunoff returning the result of a plurality Runoff voting (plurality with two
rounds).

In [4]:
def PluralityRunoff(arr_candidates, voters):
        
    dictionary = {}
    count = [0] * len(arr_candidates)
    b = False
    
    for voter in voters:
        for i in voter:
            for candidate in arr_candidates:
                #sum all votes for one candidate
                if i == candidate:
                    count[arr_candidates.index(candidate)] += voter[len(voter)-1]
                    b = True
                    
            if b == True:
                break

    for u in range(len(count)-1):
        for voter in voters:
            #create a condition, when the first candidate in one preference has minimum amout of votes
            if voter[0] == arr_candidates[np.argmin(count)]:
                a = False
                #then we go through this array of preference
                for v in voter[1:]:
                    for i in range(len(arr_candidates)):
                        #check if next candidate in the preference also exists in an array of all available candidates
                        if v == arr_candidates[i]:
                            #if it exists, add the sum of the votes for the candidate with the lowest number to the 
                            #sum of the votes of the next candidate in each preference
                            count[i] += voter[len(voter)-1]
                            a = True
                    if a == True:
                            break
                            
        #delete name of candidate (with the lowest number of votes) in the array of all available candidates                
        arr_candidates.pop(np.argmin(count))
        #delete sum of votes with index of the candidate with the lowest number of votes in the array  
        count.pop(np.argmin(count))
    
    #return name of candidate with maximum number of votes
    return arr_candidates[0]

print("According to PluralityRunoff rule, candidate -", PluralityRunoff(['a', 'b', 'c', 'd',], voters_1), "is elected")

According to PluralityRunoff rule, candidate - a is elected


4. Compute a function CondorcetVoting returning the result of the application of the Condorcet principle (the
existence of the Condorcet winner).

In [5]:
def CondorcetVoting(arr_candidates, voters):
    
    dictionary = {}
    count = [0] * len(arr_candidates)
    
    #create a cycle to go through the array with names of all candidates
    for i in range(len(arr_candidates)):
        for j in range(len(arr_candidates)):
            #send two candidates to the MajorityRule function, compare to which candidate the answer from the function is equal
            if MajorityRule(arr_candidates[i], arr_candidates[j], voters) == arr_candidates[i]:
                #if equal, then this candidate is the winner and we add 1 to the count for each win
                count[i] += 1
         
    #create dictionary with names of the candidates and sum of their votes
    for candidate in arr_candidates:
        dictionary[count[arr_candidates.index(candidate)]] = candidate

    #return candidate name with maximum value of votes
    winner = dictionary.get(max(count))
    return winner

print("According to Condorcet voting rule, candidate -", CondorcetVoting(['a', 'b', 'c', 'd'], voters_1), "is elected")

According to Condorcet voting rule, candidate - c is elected


5. Compute a function BordaVoting returning the result of the application of the Borda principle.

In [6]:
def BordaVoting(arr_candidates, voters):
    
    dictionary = {}
    count = [0] * len(arr_candidates)
    
    for voter in voters:
        count1 = 1
        for i in voter:
            for candidate in arr_candidates:
                #sum all votes for one candidate mulriple by number of votes
                if i == candidate:
                    count[arr_candidates.index(candidate)] += count1 * voter[len(voter)-1]
                    count1 += 1
            
    #create dictionary with names of the candidates and sum of their votes
    for candidate in arr_candidates:
        dictionary[count[arr_candidates.index(candidate)]] = candidate

    #return candidate name with maximum value of votes
    winner = dictionary.get(min(count))
    return winner

print("According to Borda voting rule, candidate -", BordaVoting(['a', 'b', 'c', 'd'], voters_1), "is elected")

According to Borda voting rule, candidate - b is elected


6. Elaborate an election example with n ≥ 40 and m ≥ 6 where the winner is the same for the four voting rules
Plurality, Plurality with Runoff, Condorcet Principle and Borda rules. In your example, at least 10% of voters should
have different preferences and no more than 70% of voters has the same “best candidate”.

In [8]:
def GenerateData(n, m):
    
    #create a condition, so that the function only works when the parameters are set correctly
    if n >= 40 and m >= 6:
        
        first = []
        mylist = [k for k in range(0, m)]
        data = np.empty((n, m))
        final_list = []
        list_votes = []
        list_duplicates = []
        
        array = [''] * m
        
        #create and fill an array of preferences for given candidates
        for k in range(len(array)): array[k] = "c" + str(float(k))
        
        for j in range(0, n):
            random.shuffle(mylist)
            for i in range(0, m):
                data[j,i]= str(mylist[i])
                
            df = ["c" + str(d) for d in data[j]]
            final_list.append(df)
        
        #search for the duplicate preference
        for x in final_list:
            if x not in list_votes:
                list_votes.append(x)
            elif x in list_votes:
                list_duplicates.append(x)
    
        count_1 = [0] * n
        
        #calculate votes of the sets of preference
        for candidate in range(len(list_votes)):
            count_1[candidate]=1
            for cand in range(len(list_duplicates)):
                if list_votes[candidate] == list_duplicates[cand]:
                    count_1[candidate] += 1
            
        #add amount of votes for each preference to array with preferences
        for candidate in range(len(list_votes)):
            for c1 in range(len(count_1)):
                list_votes[candidate].append(count_1[candidate])
                break
        
        #check that no more than 70% of voters has the same “best candidate”
        first = [item[0] for item in final_list]
    
        count = 0
        count = collections.Counter(first)
        duplicates = []
        duplicates = [count[y] for y in count]
    
        for z in range(len(duplicates)):
            if duplicates[z]/n > 0.7: 
                GenerateData(n, m)
        
        #check that at least 10% of voters should have different preferences
        for i in count_1:
            if i/sum(count_1) > 0.1: GenerateData(n,m)
        
        #check that the winner is the same for the four voting rules Plurality, Plurality with Runoff, Condorcet Principle and Borda rules
        if Plurality(array, list_votes) != PluralityRunoff(array, list_votes) != BordaVoting(array, list_votes) != CondorcetVoting(array, list_votes):
            GenerateData(n, m)
            
        #print the winner for each rule of the voting process
        print("According to Plurality rule, candidate -", Plurality(array, list_votes), "is elected")
        print("According to PluralityRunoff rule, candidate -", PluralityRunoff(array, list_votes), "is elected")
        print("According to Borda voting rule, candidate -", BordaVoting(array, list_votes), "is elected")
        print("According to Condorcet voting rule, candidate -", CondorcetVoting(array, list_votes), "is elected")
    
        print()
        #print the election example
        print("Generated election example is given below:")
        for candidate in list_votes:
            result = f'{str(candidate[len(candidate)-1])+ " voters: "}{candidate[:-1]}'
            print(result)
            
    else:
        return ("Please try again! n should be bigger than 40 and m should be bigger than 6" )
    
n = input("Type a number of voters (n): ")
m = input("Type a number of candidates (m): ")

GenerateData(int(n), int(m))

Type a number of voters (n): 40
Type a number of candidates (m): 6
According to Plurality rule, candidate - c4.0 is elected
According to PluralityRunoff rule, candidate - c4.0 is elected
According to Borda voting rule, candidate - c4.0 is elected
According to Condorcet voting rule, candidate - c4.0 is elected

Generated election example is given below:
1 voters: ['c0.0', 'c2.0', 'c4.0', 'c5.0', 'c3.0', 'c1.0']
1 voters: ['c5.0', 'c0.0', 'c4.0', 'c3.0', 'c2.0', 'c1.0']
1 voters: ['c1.0', 'c0.0', 'c3.0', 'c5.0', 'c4.0', 'c2.0']
1 voters: ['c4.0', 'c5.0', 'c3.0', 'c2.0', 'c0.0', 'c1.0']
1 voters: ['c1.0', 'c4.0', 'c3.0', 'c2.0', 'c5.0', 'c0.0']
1 voters: ['c2.0', 'c5.0', 'c3.0', 'c1.0', 'c4.0', 'c0.0']
1 voters: ['c2.0', 'c1.0', 'c0.0', 'c4.0', 'c5.0', 'c3.0']
1 voters: ['c5.0', 'c4.0', 'c0.0', 'c3.0', 'c1.0', 'c2.0']
1 voters: ['c1.0', 'c2.0', 'c4.0', 'c0.0', 'c3.0', 'c5.0']
1 voters: ['c2.0', 'c4.0', 'c1.0', 'c3.0', 'c5.0', 'c0.0']
1 voters: ['c4.0', 'c1.0', 'c0.0', 'c5.0', 'c2.0', 'c3.

7. Elaborate an election example with n ≥ 40 and m ≥ 6 where the winner is not the same for the four voting rules
Plurality, Plurality with Runoff, Condorcet Principle and Borda rules. In your example, at least 10% of voters should
have different preferences and no more than 70% of voters has the same “best candidate”.

In [10]:
def GenerateData7(n, m):
    
    #create a condition, so that the function only works when the parameters are set correctly
    if n >= 40 and m >= 6:
        
        first = []
        mylist = [k for k in range(0, m)]
        data = np.empty((n, m))
        final_list = []
        list_votes = []
        list_duplicates = []
        
        array = [''] * m
        
        #create and fill an array of preferences for given candidates
        for k in range(len(array)): array[k] = "c" + str(float(k))
        
        for j in range(0, n):
            random.shuffle(mylist)
            for i in range(0, m):
                data[j,i]= str(mylist[i])
                
            df = ["c" + str(d) for d in data[j]]
            final_list.append(df)
        
        #search for the duplicate preference
        for x in final_list:
            if x not in list_votes:
                list_votes.append(x)
            elif x in list_votes:
                list_duplicates.append(x)
    
        count_1 = [0] * n
        
        #calculate votes of the sets of preference
        for candidate in range(len(list_votes)):
            count_1[candidate]=1
            for cand in range(len(list_duplicates)):
                if list_votes[candidate] == list_duplicates[cand]:
                    count_1[candidate] += 1
            
        #add amount of votes for each preference to array with preferences
        for candidate in range(len(list_votes)):
            for c1 in range(len(count_1)):
                list_votes[candidate].append(count_1[candidate])
                break
        
        #check that no more than 70% of voters has the same “best candidate”
        first = [item[0] for item in final_list]
    
        count = 0
        count = collections.Counter(first)
        duplicates = []
        duplicates = [count[y] for y in count]
    
        for z in range(len(duplicates)):
            if duplicates[z]/n > 0.7: 
                GenerateData7(n, m)
        
        #check that at least 10% of voters should have different preferences
        for i in count_1:
            if i/sum(count_1) > 0.1: GenerateData7(n,m)
            
        #print the winner for each rule of the voting process
        print("According to Plurality rule, candidate -", Plurality(array, list_votes), "is elected")
        print("According to PluralityRunoff rule, candidate -", PluralityRunoff(array, list_votes), "is elected")
        print("According to Borda voting rule, candidate -", BordaVoting(array, list_votes), "is elected")
        print("According to Condorcet voting rule, candidate -", CondorcetVoting(array, list_votes), "is elected")
    
        print()
        #print the election example
        print("Generated election example is given below:")
        for candidate in list_votes:
            result = f'{str(candidate[len(candidate)-1])+ " voters: "}{candidate[:-1]}'
            print(result)
            
    else:
        return ("Please try again! n should be bigger than 40 and m should be bigger than 6" )
    
n = input("Type a number of voters (n): ")
m = input("Type a number of candidates (m): ")

GenerateData7(int(n), int(m))

Type a number of voters (n): 50
Type a number of candidates (m): 6
According to Plurality rule, candidate - c5.0 is elected
According to PluralityRunoff rule, candidate - c2.0 is elected
According to Borda voting rule, candidate - c2.0 is elected
According to Condorcet voting rule, candidate - c2.0 is elected

Generated election example is given below:
1 voters: ['c5.0', 'c0.0', 'c3.0', 'c1.0', 'c4.0', 'c2.0']
1 voters: ['c5.0', 'c3.0', 'c0.0', 'c2.0', 'c1.0', 'c4.0']
1 voters: ['c3.0', 'c4.0', 'c5.0', 'c1.0', 'c0.0', 'c2.0']
1 voters: ['c4.0', 'c2.0', 'c3.0', 'c5.0', 'c0.0', 'c1.0']
1 voters: ['c5.0', 'c1.0', 'c4.0', 'c3.0', 'c0.0', 'c2.0']
1 voters: ['c2.0', 'c0.0', 'c3.0', 'c1.0', 'c5.0', 'c4.0']
1 voters: ['c4.0', 'c0.0', 'c1.0', 'c2.0', 'c3.0', 'c5.0']
1 voters: ['c2.0', 'c0.0', 'c3.0', 'c4.0', 'c1.0', 'c5.0']
1 voters: ['c1.0', 'c0.0', 'c2.0', 'c5.0', 'c3.0', 'c4.0']
1 voters: ['c1.0', 'c0.0', 'c3.0', 'c2.0', 'c4.0', 'c5.0']
1 voters: ['c4.0', 'c0.0', 'c2.0', 'c1.0', 'c3.0', 'c5.